# Phase 1 : Physics-Informed Feature Engineering and Dataset Development 

## This notebook mainly focusses on the development of features having actual physical quantities needed for the relation in that pressure strain correlation formula (Phase 0.ipynb). All these developed features and labels will then be used to make the corresponding datasets which will be useful for later phases.

---

## DNS Dataset Description (Lee & Moser Channel Flow)

### This work uses Direct Numerical Simulation (DNS) data of fully developed turbulent plane channel flow provided by Lee & Moser (2015). DNS resolves all turbulence scales, making it an ideal high-fidelity dataset for data-driven turbulence modeling.

## 1. Mean Flow Statistics (*_mean_prof.dat)

### These files contain Reynolds-averaged (mean) quantities as a function of the wall-normal coordinate $y$.

### Typical contents:

#### - $y$ : wall-normal coordinate

#### - $\langle U \rangle(y)$ : mean streamwise velocity

#### - $\langle V \rangle(y), \langle W \rangle(y)$ : mean velocities (approximately zero)

#### - $\langle p \rangle(y)$ : mean pressure

### For fully developed plane channel flow, the mean velocity vector is
### $$ \mathbf{U} = \begin{bmatrix} 
\langle U(y) \rangle \\ 
0 \\ 
0 
\end{bmatrix}
$$
### These data are used to compute the mean velocity gradient
### $$ \frac{\mathrm{d}\langle U \rangle}{\mathrm{d}y} $$
### which is a key input feature for pressure–strain correlation modeling.

---

## 2. Velocity Fluctuation Statistics (*_vel_fluc_prof.dat)

### These files contain second-order velocity moments, defining the Reynolds stress tensor
### $$ R_{ij} = \langle u_i u_j \rangle $$

### Available components

#### - $\langle u_1 u_1 \rangle = \langle uu \rangle$

#### - $\langle u_2 u_2 \rangle = \langle vv \rangle$

#### - $\langle u_3 u_3 \rangle = \langle ww \rangle$

#### - $\langle u_1 u_2 \rangle = \langle uv \rangle$

### The Reynolds stress tensor for plane channel flow is
### $$
\mathbf{R} = \begin{bmatrix} 
\langle uu \rangle & \langle uv \rangle & 0 \\ 
\langle uv \rangle & \langle vv \rangle & 0 \\ 
0 & 0 & \langle ww \rangle 
\end{bmatrix}
$$

## Turbulent Kinetic Energy

### The turbulent kinetic energy is defined as
### $$
k = \frac{1}{2} (\langle uu \rangle + \langle vv \rangle + \langle ww \rangle)
$$

## Reynolds Stress Anisotropy Tensor

### The anisotropy tensor is defined by
### $$
b_{ij} = \frac{R_{ij}}{2k} - \frac{1}{3}\delta_{ij}
$$
### where $\delta_{ij}$ denotes the Kronecker delta.
### The independent anisotropy components used are:
### $$
{b_{11}, b_{22}, b_{33}, b_{12}}
$$

---

## 3. Reynolds Stress Transport Equation Files

### (*_RSTE_uu_prof.dat, *_RSTE_vv_prof.dat, *_RSTE_ww_prof.dat, *_RSTE_uv_prof.dat)

### These files contain the terms of the Reynolds Stress Transport Equation
### $$
\frac{\mathrm{D}R_{ij}}{\mathrm{D}t} = P_{ij} + T_{ij} - \varepsilon_{ij} + \phi_{ij}
$$
### where:
#### - $P_{ij}$ is the production term

#### - $T_{ij}$ is the transport term

#### - $\varepsilon_{ij}$ is the dissipation tensor

#### - $\phi_{ij}$ is the pressure–strain correlation

### Each file corresponds to one Reynolds stress component:
### $$
R_{11}, R_{22}, R_{33}, R_{12}
$$

---

## 4. Dissipation Rate

### The scalar dissipation rate is defined as the trace of the dissipation tensor
### $$
\varepsilon = \varepsilon_{11} + \varepsilon_{22} + \varepsilon_{33}
$$
### with
### $$
\varepsilon_{11} = D_{uu}, \quad \varepsilon_{22} = D_{vv}, \quad \varepsilon_{33} = D_{ww}
$$
### The off-diagonal component $\varepsilon_{12} = D_{uv}$ does not contribute to $\varepsilon$.

---

## 5. Pressure–Strain Correlation (Machine Learning Target)

### The pressure–strain correlation tensor is defined as
### $$
\phi_{ij} = \left\langle p \left( \frac{\partial u_i}{\partial x_j} + \frac{\partial u_j}{\partial x_i} \right) \right\rangle
$$
### For plane channel flow, the independent non-zero components are
### $$
\{ \phi_{11}, \phi_{22}, \phi_{33}, \phi_{12} \}
$$

---

## 6. Reynolds Numbers in the Dataset

### The simulations are performed at four friction Reynolds numbers
### $$Re = \frac{u \cdot L}{\nu}$$
| $Re_{\tau}$ | Flow regime          | Physical interpretation   |
| ----------- | -------------------- | ------------------------- |
| $550$       | Low Reynolds number  | Strong viscous influence  |
| $1000$      | Transitional         | Developing inertial layer |
| $2000$      | Fully turbulent      | Clear logarithmic region  |
| $5200$      | High Reynolds number | Strong scale separation   |

---

## 7. Purpose of Multiple Reynolds Numbers

### Using multiple values of $Re_{\lambda}$ enables:

#### - Cross-Reynolds-number training and testing

#### - Evaluation of model generalization

#### - Application to unseen turbulent regimes

### In the referenced study, three Reynolds numbers are used for training and one Reynolds number is used for testing, rotated across cases.

---

## Summary

#### - *_mean_prof.dat $\rightarrow \langle U \rangle,\ \dfrac{\mathrm{d}\langle U\rangle}{\mathrm{d}y}$

#### - *_vel_fluc_prof.dat $\rightarrow R_{ij},\ k,\ b_{ij}$

#### - *_RSTE_*.dat $\rightarrow \varepsilon_{ij},\ \phi_{ij}$

#### - Multiple $Re_{\tau}$ $\rightarrow$ robust, physics-consistent learning

### This DNS dataset provides a physically grounded basis for machine-learning-based pressure-strain correlation modeling.

---

## First Step : Filtering the .dat files and converting them into Pandas DataFrames

### First we will start with Re - 5200, then for other Re(s).

In [1]:
import pandas as pd       # as used in Phase 0

In [2]:
file = "../DNS (Lee and Moser 2015)/Re - 5200/LM_Channel_5200_mean_prof.dat"    # file path is huge
mean_prof_5200_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
mean_prof_5200_df.columns = [
    'y/delta',
    'y^+',
    'U',
    'dU/dy',
    'W',
    'P'
]

mean_prof_5200_df

,y/delta,y^+,U,dU/dy,W,P
0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00
1,0.000014,0.071102,0.071102,0.999986,-0.000011,-4.717272e-09
2,0.000042,0.216250,0.216244,0.999947,-0.000034,-3.851045e-07
3,0.000085,0.438384,0.438355,0.999819,-0.000069,-6.055743e-06
4,0.000143,0.740447,0.740306,0.999379,-0.000117,-4.477271e-05
...,...,...,...,...,...,...
763,0.991022,5139.336002,26.574682,0.000026,-0.006544,-4.790325e-01
764,0.993017,5149.682732,26.574923,0.000020,-0.006553,-4.789895e-01
765,0.995012,5160.029605,26.575104,0.000015,-0.006562,-4.789556e-01
766,0.997007,5170.376581,26.575224,0.000009,-0.006568,-4.789319e-01


In [3]:
file = "../DNS (Lee and Moser 2015)/Re - 5200/LM_Channel_5200_vel_fluc_prof.dat"    # file path is huge
vel_fluc_5200_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
vel_fluc_5200_df.columns = [
    'y/delta',
    'y^+',
    "u'u'",
    "v'v'",
    "w'w'",
    "u'v'",
    "u'w'",
    "v'w'",
    "k"
]

vel_fluc_5200_df

,y/delta,y^+,u'u',v'v',w'w',u'v',u'w',v'w',k
0,0.000000,0.000000,4.176503e-36,0.000000e+00,-4.685007e-10,0.000000e+00,-6.964740e-40,0.000000e+00,-2.342503e-10
1,0.000014,0.071102,1.005730e-03,4.711429e-09,4.434640e-04,-3.980482e-07,-8.647142e-08,6.027252e-11,7.245992e-04
2,0.000042,0.216250,9.271135e-03,3.846287e-07,3.959369e-03,-1.137182e-05,-8.119895e-07,1.722111e-09,6.615444e-03
3,0.000085,0.438384,3.792464e-02,6.048292e-06,1.542633e-02,-9.679785e-05,-3.408324e-06,1.466385e-08,2.667851e-02
4,0.000143,0.740447,1.075992e-01,4.471796e-05,4.099240e-02,-4.781457e-04,-9.973361e-06,7.246230e-08,7.431814e-02
...,...,...,...,...,...,...,...,...,...
763,0.991022,5139.336002,7.769150e-01,4.777459e-01,4.835921e-01,-8.868758e-03,-1.041941e-03,-4.083614e-05,8.691265e-01
764,0.993017,5149.682732,7.766527e-01,4.777049e-01,4.835047e-01,-6.897897e-03,-1.021152e-03,-3.204562e-05,8.689311e-01
765,0.995012,5160.029605,7.764571e-01,4.776738e-01,4.834375e-01,-4.927030e-03,-1.004497e-03,-2.306793e-05,8.687842e-01
766,0.997007,5170.376581,7.763274e-01,4.776528e-01,4.833922e-01,-2.956175e-03,-9.930849e-04,-1.393394e-05,8.686862e-01


In [4]:
file = "../DNS (Lee and Moser 2015)/Re - 5200/LM_Channel_5200_RSTE_uu_prof.dat"    # file path is huge
uu_5200_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
uu_5200_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

uu_5200_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,-3.660650e-08,3.991001e-01,3.587972e-18,0.0,0.399097,2.780255e-06
1,0.000014,0.071102,7.969332e-07,1.827166e-06,3.969747e-01,7.126606e-04,0.0,0.397690,3.913828e-07
2,0.000042,0.216250,2.276684e-05,5.150146e-05,3.933095e-01,1.940251e-03,0.0,0.395324,-4.761791e-07
3,0.000085,0.438384,1.937701e-04,4.352405e-04,3.888439e-01,3.271317e-03,0.0,0.392745,-8.988028e-07
4,0.000143,0.740447,9.567407e-04,2.118718e-03,3.832897e-01,4.142402e-03,0.0,0.390509,-1.939824e-06
...,...,...,...,...,...,...,...,...,...
763,0.991022,5139.336002,4.923960e-07,2.659745e-04,6.887972e-07,-1.290765e-04,0.0,0.000142,-3.548063e-06
764,0.993017,5149.682732,3.020573e-07,2.655926e-04,6.522181e-07,-1.290291e-04,0.0,0.000142,-4.077895e-06
765,0.995012,5160.029605,1.608570e-07,2.653701e-04,6.341900e-07,-1.289555e-04,0.0,0.000142,-4.364544e-06
766,0.997007,5170.376581,6.884466e-08,2.651642e-04,6.065865e-07,-1.288833e-04,0.0,0.000142,-4.600712e-06


In [5]:
file = "../DNS (Lee and Moser 2015)/Re - 5200/LM_Channel_5200_RSTE_vv_prof.dat"    # file path is huge
vv_5200_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
vv_5200_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

vv_5200_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.0,2.106418e-10,1.253449e-08,-1.319691e-17,-3.474456e-10,5.738963e-34,1.239769e-08
1,0.000014,0.071102,0.0,-1.096958e-10,1.102546e-05,-3.781768e-03,3.778122e-03,7.379397e-06,2.267119e-10
2,0.000042,0.216250,0.0,-2.408200e-09,9.426692e-05,-1.084468e-02,1.081406e-02,6.364173e-05,2.042144e-09
3,0.000085,0.438384,0.0,-7.004390e-08,3.434642e-04,-2.008729e-02,1.997922e-02,2.353171e-04,5.102091e-09
4,0.000143,0.740447,0.0,-7.241371e-07,8.322618e-04,-3.000853e-02,2.976036e-02,5.833525e-04,1.147543e-08
...,...,...,...,...,...,...,...,...,...
763,0.991022,5139.336002,0.0,1.340516e-04,8.110702e-08,6.344773e-05,-6.012056e-05,1.374763e-04,-1.637255e-08
764,0.993017,5149.682732,0.0,1.349715e-04,8.468558e-08,6.357220e-05,-6.090573e-05,1.374505e-04,2.721234e-07
765,0.995012,5160.029605,0.0,1.357888e-04,9.280268e-08,6.366270e-05,-6.158857e-05,1.374277e-04,5.280466e-07
766,0.997007,5170.376581,0.0,1.362525e-04,1.053100e-07,6.375206e-05,-6.204380e-05,1.374079e-04,6.582142e-07


In [6]:
file = "../DNS (Lee and Moser 2015)/Re - 5200/LM_Channel_5200_RSTE_ww_prof.dat"    # file path is huge
ww_5200_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
ww_5200_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

ww_5200_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,-7.914839e-09,1.787243e-01,9.608935e-18,0.0,0.178722,2.349713e-06
1,0.000014,0.071102,2.939439e-14,-8.949977e-08,1.695258e-01,3.069107e-03,0.0,0.172594,7.071124e-07
2,0.000042,0.216250,8.399050e-13,-2.803089e-06,1.520600e-01,8.904433e-03,0.0,0.160962,-3.276240e-07
3,0.000085,0.438384,7.188404e-12,-2.469032e-05,1.283627e-01,1.681597e-02,0.0,0.145155,-6.819498e-07
4,0.000143,0.740447,3.600482e-11,-1.207371e-04,1.011132e-01,2.586613e-02,0.0,0.126859,-5.302098e-07
...,...,...,...,...,...,...,...,...,...
763,0.991022,5139.336002,6.923561e-10,7.398340e-05,1.660466e-07,6.562878e-05,0.0,0.000139,7.483176e-07
764,0.993017,5149.682732,5.430964e-10,7.392432e-05,1.902241e-07,6.545689e-05,0.0,0.000139,5.726421e-07
765,0.995012,5160.029605,4.313725e-10,7.344157e-05,1.711034e-07,6.529281e-05,0.0,0.000139,-6.496262e-08
766,0.997007,5170.376581,3.633532e-10,7.297649e-05,1.849414e-07,6.513124e-05,0.0,0.000139,-6.663255e-07


In [7]:
file = "../DNS (Lee and Moser 2015)/Re - 5200/LM_Channel_5200_RSTE_uv_prof.dat"    # file path is huge
uv_5200_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
uv_5200_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

uv_5200_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,-4.675153e-10,-8.617998e-09,0.152614,-1.526136e-01,-1.048253e-18,-1.227676e-08
1,0.000014,0.071102,-4.717604e-09,-1.076644e-08,-4.765403e-04,0.152299,-1.521408e-01,-3.185665e-04,-9.050405e-10
2,0.000042,0.216250,-3.851166e-07,-8.945700e-07,-1.492446e-03,0.150752,-1.502608e-01,-1.002951e-03,3.834679e-10
3,0.000085,0.438384,-6.055153e-06,-1.389801e-05,-3.141072e-03,0.146409,-1.453759e-01,-2.127446e-03,-5.249626e-09
4,0.000143,0.740447,-4.474862e-05,-1.008900e-04,-5.495086e-03,0.137671,-1.357964e-01,-3.765984e-03,-9.818190e-09
...,...,...,...,...,...,...,...,...,...
763,0.991022,5139.336002,-1.271498e-05,-5.087908e-06,-2.318134e-08,0.000010,8.213872e-06,-2.186566e-07,5.854143e-07
764,0.993017,5149.682732,-9.783065e-06,-3.661128e-06,-1.834670e-08,0.000008,6.195740e-06,-1.683597e-07,8.899435e-07
765,0.995012,5160.029605,-6.879610e-06,-2.312025e-06,-1.236252e-08,0.000006,4.458735e-06,-1.192006e-07,1.089008e-06
766,0.997007,5170.376581,-4.070852e-06,-1.280699e-06,-6.664056e-09,0.000003,2.767982e-06,-7.030182e-08,8.688310e-07


### From the six above DataFrame we can see that, all have 768 rows in total (size aligned dataset)

### Now, as stated in the summary, we need U, dU/dy, R(ij) (R11, R22, R33, R12), b(ij) (b11, b22, b33, b12), k, phi(ij) (phi11, phi22, phi33, phi12) and $\varepsilon$ for dataset creation.
### (Total 18 columns but we will use only 4 as input variables and 1 as output variable during training/testing)

---

## Second Step : Feature Extraction and Engineering

### For mean velocity gradient dU/dy, we have:

In [ ]:
df = df.sort_values('y').reset_index(drop=True)

import numpy as np    # Numpy to the rescue

df['dU_dy'] = np.nan

# Numerical Differentiation (Central Difference)
df.loc[1:-2, 'dU_dy'] = (df.loc[2:, 'U_mean'].values - df.loc[:-3, 'U_mean'].values) / (df.loc[2:, 'y'].values - df.loc[:-3, 'y'].values)

# Forward difference at first point
df.loc[0, 'dU_dy'] = (df.loc[1, 'U_mean'] - df.loc[0, 'U_mean']) / (df.loc[1, 'y'] - df.loc[0, 'y'])

# Backward difference at last point
df.loc[len(df)-1, 'dU_dy'] = (df.loc[len(df)-1, 'U_mean'] - df.loc[len(df)-2, 'U_mean']) / (df.loc[len(df)-1, 'y'] - df.loc[len(df)-2, 'y'])

### As we can see this above code is just a dummy code which we will not run as we already have the gradients (this is just for educational purposes)

### For turbulence kinetic energy k, we have:

In [ ]:
df = df.sort_values('y').reset_index(drop=True)
df['k'] = 0.5 * (df['uu'] + df['vv'] + df['ww'])

### As we can see this above code is just a dummy code which we will not run as we already have the k values (this is just for educational purposes)

### For scaler dissipation rate $\varepsilon$, we will calculate it in one go by adding all the normal dissipative terms 

In [8]:
uu_5200_df = uu_5200_df.reset_index(drop=True)     # Ensure consistent ordering
vv_5200_df = vv_5200_df.reset_index(drop=True)
ww_5200_df = ww_5200_df.reset_index(drop=True)

epsilon_5200 = (uu_5200_df['Viscous_Dissipation'] + vv_5200_df['Viscous_Dissipation'] + ww_5200_df['Viscous_Dissipation'])  
epsilon_5200_df = pd.DataFrame({        # Converting the series to DataFrame for merging it into the final Dataset
    'y^+': uu_5200_df['y^+'] ,
    'epsilon': epsilon_5200
})

### For the Anisotropy Tensor b(ij), we will calculate it with the help of above formula with R(ij) and k

In [9]:
vel_fluc_5200_df = vel_fluc_5200_df.sort_values('y^+').reset_index(drop=True)
b_11_5200 = vel_fluc_5200_df["u'u'"] / (2.0 * vel_fluc_5200_df['k'] ) - 1.0 / 3.0  # All these four are Pandas Series
b_22_5200 = vel_fluc_5200_df["v'v'"] / (2.0 * vel_fluc_5200_df['k']) - 1.0 / 3.0
b_33_5200 = vel_fluc_5200_df["w'w'"] / (2.0 * vel_fluc_5200_df['k']) - 1.0 / 3.0
b_12_5200 = vel_fluc_5200_df["u'v'"] / (2.0 * vel_fluc_5200_df['k'])

b_5200_df = pd.DataFrame({                      # Converting the series to DataFrame for merging it into the final Dataset     
    'y^+': uu_5200_df['y^+'],
    'b_11': b_11_5200,
    'b_22': b_22_5200,
    'b_33': b_33_5200,
    'b_12': b_12_5200
})

### For Pressure-Strain Correlation phi(ij), we have 

In [10]:
phi_5200_df = pd.DataFrame({                      # Converting the series to DataFrame for merging it into the final Dataset     
    'y^+': uu_5200_df['y^+'],
    'phi_11': uu_5200_df['Pressure_Strain'],
    'phi_22': vv_5200_df['Pressure_Strain'],
    'phi_33': ww_5200_df['Pressure_Strain'],
    'phi_12': uv_5200_df['Pressure_Strain']
})

## Therefore, we got all the features which we will use in our final dataset (for Re - 5200)

---

## Third Step : Merging the features and labels to make the final dataset

In [11]:
df_5200 = (
    mean_prof_5200_df[['y/delta','y^+', 'U']]
    .merge(mean_prof_5200_df[['y^+', 'dU/dy']], on='y^+')
    .merge(vel_fluc_5200_df[['y^+', 'k']], on='y^+')
    .merge(vel_fluc_5200_df[['y^+',"u'u'","v'v'","w'w'","u'v'"]], on='y^+')
    .merge(epsilon_5200_df[['y^+', 'epsilon']], on='y^+')
    .merge(b_5200_df[['y^+', 'b_11', 'b_22', 'b_33', 'b_12']], on='y^+')
    .merge(phi_5200_df[['y^+','phi_11', 'phi_22', 'phi_33', 'phi_12']], on='y^+')
)

df_5200

,y/delta,y^+,U,dU/dy,k,u'u',v'v',w'w',u'v',epsilon,b_11,b_22,b_33,b_12,phi_11,phi_22,phi_33,phi_12
0,0.000000,0.000000,0.000000,1.000000,-2.342503e-10,4.176503e-36,0.000000e+00,-4.685007e-10,0.000000e+00,0.577819,-0.333333,-0.333333,0.666667,-0.000000,3.587972e-18,-1.319691e-17,9.608935e-18,0.152614
1,0.000014,0.071102,0.071102,0.999986,7.245992e-04,1.005730e-03,4.711429e-09,4.434640e-04,-3.980482e-07,0.570291,0.360657,-0.333330,-0.027327,-0.000275,7.126606e-04,-3.781768e-03,3.069107e-03,0.152299
2,0.000042,0.216250,0.216244,0.999947,6.615444e-03,9.271135e-03,3.846287e-07,3.959369e-03,-1.137182e-05,0.556350,0.367386,-0.333304,-0.034081,-0.000859,1.940251e-03,-1.084468e-02,8.904433e-03,0.150752
3,0.000085,0.438384,0.438355,0.999819,2.667851e-02,3.792464e-02,6.048292e-06,1.542633e-02,-9.679785e-05,0.538135,0.377438,-0.333220,-0.044218,-0.001814,3.271317e-03,-2.008729e-02,1.681597e-02,0.146409
4,0.000143,0.740447,0.740306,0.999379,7.431814e-02,1.075992e-01,4.471796e-05,4.099240e-02,-4.781457e-04,0.517952,0.390576,-0.333032,-0.057543,-0.003217,4.142402e-03,-3.000853e-02,2.586613e-02,0.137671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0.991022,5139.336002,26.574682,0.000026,8.691265e-01,7.769150e-01,4.777459e-01,4.835921e-01,-8.868758e-03,0.000418,0.113618,-0.058491,-0.055127,-0.005102,-1.290765e-04,6.344773e-05,6.562878e-05,0.000010
764,0.993017,5149.682732,26.574923,0.000020,8.689311e-01,7.766527e-01,4.777049e-01,4.835047e-01,-6.897897e-03,0.000418,0.113568,-0.058453,-0.055115,-0.003969,-1.290291e-04,6.357220e-05,6.545689e-05,0.000008
765,0.995012,5160.029605,26.575104,0.000015,8.687842e-01,7.764571e-01,4.776738e-01,4.834375e-01,-4.927030e-03,0.000418,0.113531,-0.058424,-0.055107,-0.002836,-1.289555e-04,6.366270e-05,6.529281e-05,0.000006
766,0.997007,5170.376581,26.575224,0.000009,8.686862e-01,7.763274e-01,4.776528e-01,4.833922e-01,-2.956175e-03,0.000418,0.113507,-0.058405,-0.055102,-0.001702,-1.288833e-04,6.375206e-05,6.513124e-05,0.000003


In [12]:
df_5200.to_csv("../Datasets/Re-5200.csv", index=False)    # Saving the DataFrame in the Datasets Folder (For model training)

## We got our first dataset (likewise we will do for the other Re(s))

### The furthur Re(s) will have the same number of steps and procedure, so we will just glance over the code not document it (it is just the same procedure as Re - 5200)

---

## Re - 2000

In [13]:
file = "../DNS (Lee and Moser 2015)/Re - 2000/LM_Channel_2000_mean_prof.dat"    # file path is huge
mean_prof_2000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
mean_prof_2000_df.columns = [
    'y/delta',
    'y^+',
    'U',
    'dU/dy',
    'W',
    'P'
]

mean_prof_2000_df

,y/delta,y^+,U,dU/dy,W,P
0,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000e+00
1,0.000001,0.002428,0.002428,0.999999,9.857584e-07,-6.103635e-15
2,0.000006,0.012141,0.012141,0.999994,4.928775e-06,-3.802941e-12
3,0.000017,0.033995,0.033995,0.999983,1.380050e-05,-2.321251e-10
4,0.000037,0.072846,0.072845,0.999963,2.957229e-05,-4.834114e-09
...,...,...,...,...,...,...
379,0.981425,1957.703001,24.393055,0.000150,1.353638e-02,-4.404069e-01
380,0.985552,1965.936430,24.394150,0.000116,1.358978e-02,-4.402281e-01
381,0.989680,1974.170350,24.394972,0.000083,1.363158e-02,-4.400968e-01
382,0.993808,1982.404621,24.395523,0.000050,1.365969e-02,-4.400110e-01


In [14]:
file = "../DNS (Lee and Moser 2015)/Re - 2000/LM_Channel_2000_vel_fluc_prof.dat"    # file path is huge
vel_fluc_2000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
vel_fluc_2000_df.columns = [
    'y/delta',
    'y^+',
    "u'u'",
    "v'v'",
    "w'w'",
    "u'v'",
    "u'w'",
    "v'w'",
    "k"
]

vel_fluc_2000_df

,y/delta,y^+,u'u',v'v',w'w',u'v',u'w',v'w',k
0,0.000000,0.000000,1.835759e-34,-0.000000e+00,3.147917e-34,0.000000e+00,-3.186099e-36,0.000000e+00,2.491838e-34
1,0.000001,0.002428,1.102624e-06,6.103635e-15,4.922083e-07,-1.525933e-11,3.435791e-10,-4.940951e-15,7.974160e-07
2,0.000006,0.012141,2.755853e-05,3.802941e-12,1.227540e-05,-1.909378e-09,8.598458e-09,-6.189646e-13,1.991697e-05
3,0.000017,0.033995,2.159370e-04,2.321251e-10,9.571660e-05,-4.201144e-08,6.757046e-08,-1.365310e-11,1.558269e-04
4,0.000037,0.072846,9.905815e-04,4.834114e-09,4.352952e-04,-4.150592e-07,3.115641e-07,-1.354510e-10,7.129408e-04
...,...,...,...,...,...,...,...,...,...
379,0.981425,1957.703001,7.929639e-01,4.404069e-01,4.463265e-01,-1.840350e-02,-4.632239e-03,-1.048933e-04,8.398487e-01
380,0.985552,1965.936430,7.917362e-01,4.402281e-01,4.459160e-01,-1.429979e-02,-4.636188e-03,-8.274932e-05,8.389401e-01
381,0.989680,1974.170350,7.908117e-01,4.400968e-01,4.456111e-01,-1.020632e-02,-4.638431e-03,-5.970856e-05,8.382598e-01
382,0.993808,1982.404621,7.901871e-01,4.400110e-01,4.454122e-01,-6.120674e-03,-4.640965e-03,-3.668884e-05,8.378052e-01


In [15]:
file = "../DNS (Lee and Moser 2015)/Re - 2000/LM_Channel_2000_RSTE_uu_prof.dat"    # file path is huge
uu_2000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
uu_2000_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

uu_2000_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,2.985328e-14,0.374026,7.728304e-20,0.0,0.374026,3.873988e-10
1,0.000001,0.002428,3.051865e-11,6.476112e-11,0.373955,2.340631e-05,0.0,0.373979,-1.052930e-08
2,0.000006,0.012141,3.818735e-09,8.102487e-09,0.373677,1.162183e-04,0.0,0.373794,-4.274095e-08
3,0.000017,0.033995,8.402151e-08,1.781187e-07,0.373068,3.203132e-04,0.0,0.373389,-6.991362e-08
4,0.000037,0.072846,8.300884e-07,1.757100e-06,0.372037,6.671484e-04,0.0,0.372707,-4.301516e-08
...,...,...,...,...,...,...,...,...,...
379,0.981425,1957.703001,5.581680e-06,6.949467e-04,0.000005,-3.351067e-04,0.0,0.000375,-4.951457e-06
380,0.985552,1965.936430,3.428675e-06,6.971126e-04,0.000004,-3.338951e-04,0.0,0.000375,-3.463923e-06
381,0.989680,1974.170350,1.833620e-06,7.002113e-04,0.000004,-3.329733e-04,0.0,0.000374,-8.239614e-07
382,0.993808,1982.404621,7.716640e-07,7.019827e-04,0.000004,-3.324774e-04,0.0,0.000374,6.601633e-07


In [16]:
file = "../DNS (Lee and Moser 2015)/Re - 2000/LM_Channel_2000_RSTE_vv_prof.dat"    # file path is huge
vv_2000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
vv_2000_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

vv_2000_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.0,1.151339e-17,5.166564e-12,-2.611154e-19,9.698288e-15,6.424511e-36,5.176273e-12
1,0.000001,0.002428,0.0,-8.436840e-18,1.241103e-08,-1.243805e-04,1.243764e-04,8.274448e-09,9.572794e-13
2,0.000006,0.012141,0.0,-1.527566e-15,3.086541e-07,-6.194981e-04,6.193954e-04,2.059028e-07,7.257226e-13
3,0.000017,0.033995,0.0,-2.612715e-13,2.391911e-06,-1.719539e-03,1.718745e-03,1.597522e-06,1.768711e-11
4,0.000037,0.072846,0.0,-1.139122e-11,1.075879e-05,-3.628024e-03,3.624466e-03,7.201028e-06,1.146233e-10
...,...,...,...,...,...,...,...,...,...
379,0.981425,1957.703001,0.0,3.388466e-04,7.627333e-07,1.570367e-04,-1.456346e-04,3.510138e-04,-2.412962e-09
380,0.985552,1965.936430,0.0,3.371910e-04,6.981504e-07,1.565645e-04,-1.443398e-04,3.506662e-04,-5.524397e-07
381,0.989680,1974.170350,0.0,3.350790e-04,6.493985e-07,1.562061e-04,-1.427931e-04,3.504077e-04,-1.266308e-06
382,0.993808,1982.404621,0.0,3.333098e-04,6.169321e-07,1.560654e-04,-1.416588e-04,3.502326e-04,-1.899228e-06


In [17]:
file = "../DNS (Lee and Moser 2015)/Re - 2000/LM_Channel_2000_RSTE_ww_prof.dat"    # file path is huge
ww_2000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
ww_2000_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

ww_2000_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,2.579147e-13,0.167055,1.838323e-19,0.0,0.167055,1.431182e-09
1,0.000001,0.002428,5.358531e-18,-3.738649e-12,0.166752,1.009742e-04,0.0,0.166853,-2.330328e-08
2,0.000006,0.012141,6.710058e-16,-4.504811e-10,0.165545,5.032799e-04,0.0,0.166048,-9.281454e-08
3,0.000017,0.033995,1.478825e-14,-1.002576e-08,0.162858,1.399226e-03,0.0,0.164257,-1.379312e-07
4,0.000037,0.072846,1.465080e-13,-1.007894e-07,0.158179,2.960875e-03,0.0,0.161140,-3.157270e-08
...,...,...,...,...,...,...,...,...,...
379,0.981425,1957.703001,-2.939000e-09,1.782763e-04,0.000002,1.780701e-04,0.0,0.000359,-1.092311e-06
380,0.985552,1965.936430,-2.875231e-09,1.782669e-04,0.000002,1.773306e-04,0.0,0.000359,-1.402988e-06
381,0.989680,1974.170350,-2.393130e-09,1.778550e-04,0.000002,1.767672e-04,0.0,0.000358,-2.034722e-06
382,0.993808,1982.404621,-1.498523e-09,1.769947e-04,0.000001,1.764119e-04,0.0,0.000358,-3.062143e-06


In [18]:
file = "../DNS (Lee and Moser 2015)/Re - 2000/LM_Channel_2000_RSTE_uv_prof.dat"    # file path is huge
uv_2000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
uv_2000_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

uv_2000_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,3.171617e-15,-3.420992e-11,0.136460,-0.136460,-1.671989e-20,-3.416079e-11
1,0.000001,0.002428,-6.100985e-15,-1.502587e-14,-1.553161e-05,0.136454,-0.136449,-1.035534e-05,-1.506349e-11
2,0.000006,0.012141,-3.801273e-12,-8.168438e-12,-7.781881e-05,0.136427,-0.136401,-5.190256e-05,-1.728771e-10
3,0.000017,0.033995,-2.320208e-10,-4.982066e-10,-2.188989e-04,0.136347,-0.136274,-1.461150e-04,-9.925857e-10
4,0.000037,0.072846,-4.831851e-09,-1.035534e-08,-4.728485e-04,0.136143,-0.135986,-3.160654e-04,-2.532884e-09
...,...,...,...,...,...,...,...,...,...
379,0.981425,1957.703001,-6.592431e-05,-3.149261e-05,-1.750534e-07,0.000044,0.000051,-2.094332e-06,-6.183226e-07
380,0.985552,1965.936430,-5.120330e-05,-2.259775e-05,-1.556053e-07,0.000036,0.000037,-1.637015e-06,2.862001e-07
381,0.989680,1974.170350,-3.669232e-05,-1.452316e-05,-1.188709e-07,0.000027,0.000024,-1.186682e-06,4.107907e-07
382,0.993808,1982.404621,-2.212651e-05,-7.762304e-06,-7.218987e-08,0.000017,0.000013,-7.258401e-07,2.908572e-07


---

In [19]:
uu_2000_df = uu_2000_df.reset_index(drop=True)     # Ensure consistent ordering
vv_2000_df = vv_2000_df.reset_index(drop=True)
ww_2000_df = ww_2000_df.reset_index(drop=True)

epsilon_2000 = (uu_2000_df['Viscous_Dissipation'] + vv_2000_df['Viscous_Dissipation'] + ww_2000_df['Viscous_Dissipation'])  
epsilon_2000_df = pd.DataFrame({        # Converting the series to DataFrame for merging it into the final Dataset
    'y^+': uu_2000_df['y^+'] ,
    'epsilon': epsilon_2000
})

In [20]:
vel_fluc_2000_df = vel_fluc_2000_df.sort_values('y^+').reset_index(drop=True)
b_11_2000 = vel_fluc_2000_df["u'u'"] / (2.0 * vel_fluc_2000_df['k'] ) - 1.0 / 3.0  # All these four are Pandas Series
b_22_2000 = vel_fluc_2000_df["v'v'"] / (2.0 * vel_fluc_2000_df['k']) - 1.0 / 3.0
b_33_2000 = vel_fluc_2000_df["w'w'"] / (2.0 * vel_fluc_2000_df['k']) - 1.0 / 3.0
b_12_2000 = vel_fluc_2000_df["u'v'"] / (2.0 * vel_fluc_2000_df['k'])

b_2000_df = pd.DataFrame({                      # Converting the series to DataFrame for merging it into the final Dataset     
    'y^+': uu_2000_df['y^+'],
    'b_11': b_11_2000,
    'b_22': b_22_2000,
    'b_33': b_33_2000,
    'b_12': b_12_2000
})

In [21]:
phi_2000_df = pd.DataFrame({                      # Converting the series to DataFrame for merging it into the final Dataset     
    'y^+': uu_2000_df['y^+'],
    'phi_11': uu_2000_df['Pressure_Strain'],
    'phi_22': vv_2000_df['Pressure_Strain'],
    'phi_33': ww_2000_df['Pressure_Strain'],
    'phi_12': uv_2000_df['Pressure_Strain']
})

---

In [22]:
df_2000 = (
    mean_prof_2000_df[['y/delta','y^+', 'U']]
    .merge(mean_prof_2000_df[['y^+', 'dU/dy']], on='y^+')
    .merge(vel_fluc_2000_df[['y^+', 'k']], on='y^+')
    .merge(vel_fluc_2000_df[['y^+',"u'u'","v'v'","w'w'","u'v'"]], on='y^+')
    .merge(epsilon_2000_df[['y^+', 'epsilon']], on='y^+')
    .merge(b_2000_df[['y^+', 'b_11', 'b_22', 'b_33', 'b_12']], on='y^+')
    .merge(phi_2000_df[['y^+','phi_11', 'phi_22', 'phi_33', 'phi_12']], on='y^+')
)

df_2000

,y/delta,y^+,U,dU/dy,k,u'u',v'v',w'w',u'v',epsilon,b_11,b_22,b_33,b_12,phi_11,phi_22,phi_33,phi_12
0,0.000000,0.000000,0.000000,1.000000,2.491838e-34,1.835759e-34,-0.000000e+00,3.147917e-34,0.000000e+00,0.541080,0.035021,-0.333333,0.298312,0.000000,7.728304e-20,-2.611154e-19,1.838323e-19,0.136460
1,0.000001,0.002428,0.002428,0.999999,7.974160e-07,1.102624e-06,6.103635e-15,4.922083e-07,-1.525933e-11,0.540831,0.358040,-0.333333,-0.024706,-0.000010,2.340631e-05,-1.243805e-04,1.009742e-04,0.136454
2,0.000006,0.012141,0.012141,0.999994,1.991697e-05,2.755853e-05,3.802941e-12,1.227540e-05,-1.909378e-09,0.539842,0.358502,-0.333333,-0.025169,-0.000048,1.162183e-04,-6.194981e-04,5.032799e-04,0.136427
3,0.000017,0.033995,0.033995,0.999983,1.558269e-04,2.159370e-04,2.321251e-10,9.571660e-05,-4.201144e-08,0.537648,0.359541,-0.333333,-0.026209,-0.000135,3.203132e-04,-1.719539e-03,1.399226e-03,0.136347
4,0.000037,0.072846,0.072845,0.999963,7.129408e-04,9.905815e-04,4.834114e-09,4.352952e-04,-4.150592e-07,0.533854,0.361382,-0.333330,-0.028052,-0.000291,6.671484e-04,-3.628024e-03,2.960875e-03,0.136143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,0.981425,1957.703001,24.393055,0.000150,8.398487e-01,7.929639e-01,4.404069e-01,4.463265e-01,-1.840350e-02,0.001085,0.138754,-0.071139,-0.067615,-0.010956,-3.351067e-04,1.570367e-04,1.780701e-04,0.000044
380,0.985552,1965.936430,24.394150,0.000116,8.389401e-01,7.917362e-01,4.402281e-01,4.459160e-01,-1.429979e-02,0.001084,0.138534,-0.070962,-0.067572,-0.008523,-3.338951e-04,1.565645e-04,1.773306e-04,0.000036
381,0.989680,1974.170350,24.394972,0.000083,8.382598e-01,7.908117e-01,4.400968e-01,4.456111e-01,-1.020632e-02,0.001083,0.138365,-0.070827,-0.067538,-0.006088,-3.329733e-04,1.562061e-04,1.767672e-04,0.000027
382,0.993808,1982.404621,24.395523,0.000050,8.378052e-01,7.901871e-01,4.400110e-01,4.454122e-01,-6.120674e-03,0.001082,0.138248,-0.070736,-0.067512,-0.003653,-3.324774e-04,1.560654e-04,1.764119e-04,0.000017


In [23]:
df_2000.to_csv("../Datasets/Re-2000.csv", index=False)    # Saving the DataFrame in the Datasets Folder (For model training)

---

## Re - 1000

In [24]:
file = "../DNS (Lee and Moser 2015)/Re - 1000/LM_Channel_1000_mean_prof.dat"    # file path is huge
mean_prof_1000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
mean_prof_1000_df.columns = [
    'y/delta',
    'y^+',
    'U',
    'dU/dy',
    'W',
    'P'
]

mean_prof_1000_df

,y/delta,y^+,U,dU/dy,W,P
0,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000e+00
1,0.000003,0.002766,0.002766,0.999997,-1.990170e-07,-9.232769e-15
2,0.000014,0.013829,0.013828,0.999986,-9.950769e-07,-5.750533e-12
3,0.000039,0.038719,0.038719,0.999961,-2.786178e-06,-3.507267e-10
4,0.000083,0.082969,0.082966,0.999916,-5.970244e-06,-7.293990e-09
...,...,...,...,...,...,...
251,0.972011,972.508740,22.586192,0.000481,-8.974446e-04,-4.101139e-01
252,0.978230,978.730479,22.588854,0.000375,-7.422379e-04,-4.096578e-01
253,0.984449,984.953062,22.590856,0.000268,-6.133027e-04,-4.093194e-01
254,0.990669,991.176247,22.592194,0.000161,-5.199235e-04,-4.090965e-01


In [25]:
file = "../DNS (Lee and Moser 2015)/Re - 1000/LM_Channel_1000_vel_fluc_prof.dat"    # file path is huge
vel_fluc_1000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
vel_fluc_1000_df.columns = [
    'y/delta',
    'y^+',
    "u'u'",
    "v'v'",
    "w'w'",
    "u'v'",
    "u'w'",
    "v'w'",
    "k"
]

vel_fluc_1000_df

,y/delta,y^+,u'u',v'v',w'w',u'v',u'w',v'w',k
0,0.000000,0.000000,1.509945e-34,0.000000e+00,2.849189e-34,0.000000e+00,-4.268949e-37,0.000000e+00,2.179567e-34
1,0.000003,0.002766,1.341024e-06,9.232769e-15,5.838998e-07,-2.162830e-11,-4.927590e-10,-8.961229e-16,9.624621e-07
2,0.000014,0.013829,3.351622e-05,5.750533e-12,1.455714e-05,-2.706426e-09,-1.233396e-08,-1.146319e-13,2.403668e-05
3,0.000039,0.038719,2.626071e-04,3.507267e-10,1.134218e-04,-5.955459e-08,-9.696007e-08,-2.641325e-12,1.880147e-04
4,0.000083,0.082969,1.204592e-03,7.293990e-09,5.151240e-04,-5.884895e-07,-4.473114e-07,-2.803062e-11,8.598615e-04
...,...,...,...,...,...,...,...,...,...
251,0.972011,972.508740,7.078715e-01,4.101139e-01,4.085809e-01,-2.737912e-02,-2.637580e-04,-1.703285e-04,7.632831e-01
252,0.978230,978.730479,7.053529e-01,4.096578e-01,4.077929e-01,-2.128724e-02,-2.964841e-04,-1.527840e-04,7.614018e-01
253,0.984449,984.953062,7.034512e-01,4.093194e-01,4.071979e-01,-1.520128e-02,-3.186349e-04,-1.204586e-04,7.599843e-01
254,0.990669,991.176247,7.021769e-01,4.090965e-01,4.067966e-01,-9.118776e-03,-3.374968e-04,-7.666176e-05,7.590350e-01


In [26]:
file = "../DNS (Lee and Moser 2015)/Re - 1000/LM_Channel_1000_RSTE_uu_prof.dat"    # file path is huge
uu_1000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
uu_1000_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

uu_1000_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.0,7.196544e-17,1.757295e-12,6.205140e-19,-1.426058e-14,6.371691e-36,1.743107e-12
1,0.000003,0.002766,0.0,-4.648669e-17,1.447622e-08,-1.282171e-04,1.282123e-04,9.651976e-09,3.394755e-13
2,0.000014,0.013829,0.0,-2.614208e-15,3.598269e-07,-6.383058e-04,6.381860e-04,2.400571e-07,-1.904724e-13
3,0.000039,0.038719,0.0,-4.550338e-13,2.784901e-06,-1.769869e-03,1.768944e-03,1.860369e-06,-9.177488e-13
4,0.000083,0.082969,0.0,-1.977796e-11,1.249818e-05,-3.727190e-03,3.723061e-03,8.368929e-06,-7.164589e-12
...,...,...,...,...,...,...,...,...,...
251,0.972011,972.508740,0.0,6.612596e-04,3.108234e-06,2.723624e-04,-2.782519e-04,6.581406e-04,3.377680e-07
252,0.978230,978.730479,0.0,6.638010e-04,3.042964e-06,2.706951e-04,-2.798949e-04,6.567223e-04,9.219791e-07
253,0.984449,984.953062,0.0,6.659344e-04,2.985138e-06,2.697867e-04,-2.815060e-04,6.556626e-04,1.537639e-06
254,0.990669,991.176247,0.0,6.670717e-04,2.896875e-06,2.691509e-04,-2.822168e-04,6.549535e-04,1.949160e-06


In [27]:
file = "../DNS (Lee and Moser 2015)/Re - 1000/LM_Channel_1000_RSTE_vv_prof.dat"    # file path is huge
vv_1000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
vv_1000_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

vv_1000_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.0,7.196544e-17,1.757295e-12,6.205140e-19,-1.426058e-14,6.371691e-36,1.743107e-12
1,0.000003,0.002766,0.0,-4.648669e-17,1.447622e-08,-1.282171e-04,1.282123e-04,9.651976e-09,3.394755e-13
2,0.000014,0.013829,0.0,-2.614208e-15,3.598269e-07,-6.383058e-04,6.381860e-04,2.400571e-07,-1.904724e-13
3,0.000039,0.038719,0.0,-4.550338e-13,2.784901e-06,-1.769869e-03,1.768944e-03,1.860369e-06,-9.177488e-13
4,0.000083,0.082969,0.0,-1.977796e-11,1.249818e-05,-3.727190e-03,3.723061e-03,8.368929e-06,-7.164589e-12
...,...,...,...,...,...,...,...,...,...
251,0.972011,972.508740,0.0,6.612596e-04,3.108234e-06,2.723624e-04,-2.782519e-04,6.581406e-04,3.377680e-07
252,0.978230,978.730479,0.0,6.638010e-04,3.042964e-06,2.706951e-04,-2.798949e-04,6.567223e-04,9.219791e-07
253,0.984449,984.953062,0.0,6.659344e-04,2.985138e-06,2.697867e-04,-2.815060e-04,6.556626e-04,1.537639e-06
254,0.990669,991.176247,0.0,6.670717e-04,2.896875e-06,2.691509e-04,-2.822168e-04,6.549535e-04,1.949160e-06


In [28]:
file = "../DNS (Lee and Moser 2015)/Re - 1000/LM_Channel_1000_RSTE_ww_prof.dat"    # file path is huge
ww_1000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
ww_1000_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

ww_1000_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,3.098800e-13,0.152773,-4.561514e-19,0.0,0.152774,-2.450606e-08
1,0.000003,0.002766,3.389282e-18,-5.223672e-12,0.152458,1.049876e-04,0.0,0.152563,-1.570448e-07
2,0.000014,0.013829,4.240557e-16,-6.332092e-10,0.151203,5.230770e-04,0.0,0.151726,-5.949375e-07
3,0.000039,0.038719,9.330522e-15,-1.408842e-08,0.148413,1.452984e-03,0.0,0.149867,-8.957605e-07
4,0.000083,0.082969,9.224687e-14,-1.415668e-07,0.143568,3.069844e-03,0.0,0.146638,-3.879726e-07
...,...,...,...,...,...,...,...,...,...
251,0.972011,972.508740,7.649434e-09,3.501988e-04,0.000005,3.274557e-04,0.0,0.000681,2.357976e-06
252,0.978230,978.730479,6.129398e-09,3.535271e-04,0.000005,3.246102e-04,0.0,0.000679,4.183312e-06
253,0.984449,984.953062,3.886569e-09,3.569478e-04,0.000005,3.221645e-04,0.0,0.000678,6.583653e-06
254,0.990669,991.176247,1.949793e-09,3.598792e-04,0.000005,3.205214e-04,0.0,0.000677,8.964934e-06


In [29]:
file = "../DNS (Lee and Moser 2015)/Re - 1000/LM_Channel_1000_RSTE_uv_prof.dat"    # file path is huge
uv_1000_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
uv_1000_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

uv_1000_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,7.345118e-15,-1.310896e-10,0.119053,-0.119053,5.408973e-20,-1.303219e-10
1,0.000003,0.002766,-9.232743e-15,-2.256204e-14,-1.696967e-05,0.119046,-0.119040,-1.131414e-05,-3.847924e-11
2,0.000014,0.013829,-5.750453e-12,-1.129080e-11,-8.503254e-05,0.119013,-0.118984,-5.671432e-05,-3.026233e-10
3,0.000039,0.038719,-3.507131e-10,-6.880999e-10,-2.392463e-04,0.118920,-0.118841,-1.597004e-04,-2.011660e-09
4,0.000083,0.082969,-7.293381e-09,-1.427677e-08,-5.170139e-04,0.118690,-0.118518,-3.456041e-04,-6.412842e-09
...,...,...,...,...,...,...,...,...,...
251,0.972011,972.508740,-1.971750e-04,-7.928397e-05,-2.106512e-07,0.000118,0.000145,-9.742558e-06,-3.620672e-06
252,0.978230,978.730479,-1.536088e-04,-6.175646e-05,-1.793614e-07,0.000092,0.000112,-7.597600e-06,-3.974763e-06
253,0.984449,984.953062,-1.098639e-04,-4.360200e-05,-9.605039e-08,0.000065,0.000080,-5.449046e-06,-3.070864e-06
254,0.990669,991.176247,-6.598892e-05,-2.534047e-05,-8.799991e-08,0.000039,0.000047,-3.288776e-06,-1.499838e-06


---

In [30]:
uu_1000_df = uu_1000_df.reset_index(drop=True)     # Ensure consistent ordering
vv_1000_df = vv_1000_df.reset_index(drop=True)
ww_1000_df = ww_1000_df.reset_index(drop=True)

epsilon_1000 = (uu_1000_df['Viscous_Dissipation'] + vv_1000_df['Viscous_Dissipation'] + ww_1000_df['Viscous_Dissipation'])  
epsilon_1000_df = pd.DataFrame({        # Converting the series to DataFrame for merging it into the final Dataset
    'y^+': uu_1000_df['y^+'] ,
    'epsilon': epsilon_1000
})

In [31]:
vel_fluc_1000_df = vel_fluc_1000_df.sort_values('y^+').reset_index(drop=True)
b_11_1000 = vel_fluc_1000_df["u'u'"] / (2.0 * vel_fluc_1000_df['k'] ) - 1.0 / 3.0  # All these four are Pandas Series
b_22_1000 = vel_fluc_1000_df["v'v'"] / (2.0 * vel_fluc_1000_df['k']) - 1.0 / 3.0
b_33_1000 = vel_fluc_1000_df["w'w'"] / (2.0 * vel_fluc_1000_df['k']) - 1.0 / 3.0
b_12_1000 = vel_fluc_1000_df["u'v'"] / (2.0 * vel_fluc_1000_df['k'])

b_1000_df = pd.DataFrame({                      # Converting the series to DataFrame for merging it into the final Dataset     
    'y^+': uu_1000_df['y^+'],
    'b_11': b_11_1000,
    'b_22': b_22_1000,
    'b_33': b_33_1000,
    'b_12': b_12_1000
})

In [32]:
phi_1000_df = pd.DataFrame({                      # Converting the series to DataFrame for merging it into the final Dataset     
    'y^+': uu_1000_df['y^+'],
    'phi_11': uu_1000_df['Pressure_Strain'],
    'phi_22': vv_1000_df['Pressure_Strain'],
    'phi_33': ww_1000_df['Pressure_Strain'],
    'phi_12': uv_1000_df['Pressure_Strain']
})

---

In [33]:
df_1000 = (
    mean_prof_1000_df[['y/delta','y^+', 'U']]
    .merge(mean_prof_1000_df[['y^+', 'dU/dy']], on='y^+')
    .merge(vel_fluc_1000_df[['y^+', 'k']], on='y^+')
    .merge(vel_fluc_1000_df[['y^+',"u'u'","v'v'","w'w'","u'v'"]], on='y^+')
    .merge(epsilon_1000_df[['y^+', 'epsilon']], on='y^+')
    .merge(b_1000_df[['y^+', 'b_11', 'b_22', 'b_33', 'b_12']], on='y^+')
    .merge(phi_1000_df[['y^+','phi_11', 'phi_22', 'phi_33', 'phi_12']], on='y^+')
)

df_1000

,y/delta,y^+,U,dU/dy,k,u'u',v'v',w'w',u'v',epsilon,b_11,b_22,b_33,b_12,phi_11,phi_22,phi_33,phi_12
0,0.000000,0.000000,0.000000,1.000000,2.179567e-34,1.509945e-34,0.000000e+00,2.849189e-34,0.000000e+00,0.152774,0.013053,-0.333333,0.320280,0.000000,6.205140e-19,6.205140e-19,-4.561514e-19,0.119053
1,0.000003,0.002766,0.002766,0.999997,9.624621e-07,1.341024e-06,9.232769e-15,5.838998e-07,-2.162830e-11,0.152563,0.363330,-0.333333,-0.029997,-0.000011,-1.282171e-04,-1.282171e-04,1.049876e-04,0.119046
2,0.000014,0.013829,0.013828,0.999986,2.403668e-05,3.351622e-05,5.750533e-12,1.455714e-05,-2.706426e-09,0.151727,0.363856,-0.333333,-0.030522,-0.000056,-6.383058e-04,-6.383058e-04,5.230770e-04,0.119013
3,0.000039,0.038719,0.038719,0.999961,1.880147e-04,2.626071e-04,3.507267e-10,1.134218e-04,-5.955459e-08,0.149870,0.365035,-0.333332,-0.031703,-0.000158,-1.769869e-03,-1.769869e-03,1.452984e-03,0.118920
4,0.000083,0.082969,0.082966,0.999916,8.598615e-04,1.204592e-03,7.293990e-09,5.151240e-04,-5.884895e-07,0.146655,0.367123,-0.333329,-0.033794,-0.000342,-3.727190e-03,-3.727190e-03,3.069844e-03,0.118690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0.972011,972.508740,22.586192,0.000481,7.632831e-01,7.078715e-01,4.101139e-01,4.085809e-01,-2.737912e-02,0.001997,0.130368,-0.064682,-0.065686,-0.017935,2.723624e-04,2.723624e-04,3.274557e-04,0.000118
252,0.978230,978.730479,22.588854,0.000375,7.614018e-01,7.053529e-01,4.096578e-01,4.077929e-01,-2.128724e-02,0.001992,0.129860,-0.064318,-0.065542,-0.013979,2.706951e-04,2.706951e-04,3.246102e-04,0.000092
253,0.984449,984.953062,22.590856,0.000268,7.599843e-01,7.034512e-01,4.093194e-01,4.071979e-01,-1.520128e-02,0.001989,0.129473,-0.064039,-0.065434,-0.010001,2.697867e-04,2.697867e-04,3.221645e-04,0.000065
254,0.990669,991.176247,22.592194,0.000161,7.590350e-01,7.021769e-01,4.090965e-01,4.067966e-01,-9.118776e-03,0.001987,0.129212,-0.063849,-0.065364,-0.006007,2.691509e-04,2.691509e-04,3.205214e-04,0.000039


In [34]:
df_1000.to_csv("../Datasets/Re-1000.csv", index=False)    # Saving the DataFrame in the Datasets Folder (For model training)

---

## Re - 550

In [35]:
file = "../DNS (Lee and Moser 2015)/Re - 550/LM_Channel_0550_mean_prof.dat"    # file path is huge
mean_prof_550_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
mean_prof_550_df.columns = [
    'y/delta',
    'y^+',
    'U',
    'dU/dy',
    'W',
    'P'
]

mean_prof_550_df

,y/delta,y^+,U,dU/dy,W,P
0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00
1,0.000005,0.002696,0.002696,0.999995,0.000001,-7.394299e-15
2,0.000025,0.013479,0.013478,0.999975,0.000007,-4.606107e-12
3,0.000069,0.037739,0.037738,0.999931,0.000021,-2.810221e-10
4,0.000149,0.080868,0.080862,0.999851,0.000044,-5.847961e-09
...,...,...,...,...,...,...
187,0.962515,523.123032,20.988490,0.001147,-0.003444,-4.032170e-01
188,0.970842,527.648902,20.993123,0.000900,-0.003569,-4.023527e-01
189,0.979172,532.175872,20.996627,0.000647,-0.003671,-4.016998e-01
190,0.987502,536.703628,20.998979,0.000391,-0.003745,-4.012624e-01


In [36]:
file = "../DNS (Lee and Moser 2015)/Re - 550/LM_Channel_0550_vel_fluc_prof.dat"    # file path is huge
vel_fluc_550_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
vel_fluc_550_df.columns = [
    'y/delta',
    'y^+',
    "u'u'",
    "v'v'",
    "w'w'",
    "u'v'",
    "u'w'",
    "v'w'",
    "k"
]

vel_fluc_550_df

,y/delta,y^+,u'u',v'v',w'w',u'v',u'w',v'w',k
0,0.000000,0.000000,1.458397e-34,0.000000e+00,2.948179e-34,0.000000e+00,-5.537037e-37,0.000000e+00,2.203288e-34
1,0.000005,0.002696,1.189617e-06,7.394295e-15,4.932175e-07,-1.910621e-11,6.905026e-10,-2.450116e-14,8.414171e-07
2,0.000025,0.013479,2.973260e-05,4.606104e-12,1.229710e-05,-2.390467e-09,1.727014e-08,-3.061344e-12,2.101485e-05
3,0.000069,0.037739,2.329725e-04,2.810219e-10,9.582685e-05,-5.258512e-08,1.355344e-07,-6.713858e-11,1.643998e-04
4,0.000149,0.080868,1.068751e-03,5.847957e-09,4.353315e-04,-5.193396e-07,6.234818e-07,-6.594588e-10,7.520443e-04
...,...,...,...,...,...,...,...,...,...
187,0.962515,523.123032,6.359682e-01,4.032170e-01,3.970381e-01,-3.616980e-02,4.498235e-03,3.223569e-05,7.181116e-01
188,0.970842,527.648902,6.319254e-01,4.023528e-01,3.953724e-01,-2.812336e-02,4.530165e-03,1.912388e-05,7.148253e-01
189,0.979172,532.175872,6.288644e-01,4.016998e-01,3.940958e-01,-2.008433e-02,4.564878e-03,1.240504e-05,7.123300e-01
190,0.987502,536.703628,6.268089e-01,4.012624e-01,3.932294e-01,-1.204935e-02,4.600034e-03,7.850369e-06,7.106503e-01


In [37]:
file = "../DNS (Lee and Moser 2015)/Re - 550/LM_Channel_0550_RSTE_uu_prof.dat"    # file path is huge
uu_550_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
uu_550_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

uu_550_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,4.393176e-14,0.327414,-1.208084e-19,0.0,0.327414,-5.821610e-09
1,0.000005,0.002696,3.821290e-11,6.998686e-11,0.327356,1.922750e-05,0.0,0.327375,-1.294796e-07
2,0.000025,0.013479,4.780899e-09,8.754966e-09,0.327126,9.539165e-05,0.0,0.327222,-5.151350e-07
3,0.000069,0.037739,1.051648e-07,1.923582e-07,0.326627,2.624249e-04,0.0,0.326891,-8.347730e-07
4,0.000149,0.080868,1.038542e-06,1.895821e-06,0.325791,5.447250e-04,0.0,0.326339,-4.745271e-07
...,...,...,...,...,...,...,...,...,...
187,0.962515,523.123032,8.289483e-05,2.124887e-03,0.000048,-8.910478e-04,0.0,0.001364,3.034964e-07
188,0.970842,527.648902,5.048580e-05,2.138233e-03,0.000048,-8.737131e-04,0.0,0.001359,4.171116e-06
189,0.979172,532.175872,2.585030e-05,2.149725e-03,0.000049,-8.604931e-04,0.0,0.001355,9.212543e-06
190,0.987502,536.703628,9.240299e-06,2.158689e-03,0.000050,-8.516028e-04,0.0,0.001353,1.378543e-05


In [38]:
file = "../DNS (Lee and Moser 2015)/Re - 550/LM_Channel_0550_RSTE_vv_prof.dat"    # file path is huge
vv_550_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
vv_550_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

vv_550_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.0,5.655541e-18,7.172221e-13,4.707927e-19,-2.536923e-14,6.505340e-36,6.918590e-13
1,0.000005,0.002696,0.0,-5.470910e-18,1.220340e-08,-1.099680e-04,1.099639e-04,8.136612e-09,1.530401e-13
2,0.000025,0.013479,0.0,-1.902769e-15,3.034158e-07,-5.475386e-04,5.474376e-04,2.024156e-07,-1.205876e-12
3,0.000069,0.037739,0.0,-3.243173e-13,2.349725e-06,-1.518717e-03,1.517937e-03,1.569506e-06,-1.893811e-11
4,0.000149,0.080868,0.0,-1.412417e-11,1.055674e-05,-3.200267e-03,3.196778e-03,7.067346e-06,-1.259613e-10
...,...,...,...,...,...,...,...,...,...
187,0.962515,523.123032,0.0,1.158195e-03,1.006463e-05,3.864793e-04,-4.160667e-04,1.133824e-03,4.848453e-06
188,0.970842,527.648902,0.0,1.154367e-03,1.032845e-05,3.770942e-04,-4.088142e-04,1.129215e-03,3.760007e-06
189,0.979172,532.175872,0.0,1.149186e-03,1.052848e-05,3.700933e-04,-4.018030e-04,1.125773e-03,2.231675e-06
190,0.987502,536.703628,0.0,1.144035e-03,1.063826e-05,3.654238e-04,-3.959203e-04,1.123502e-03,6.749099e-07


In [39]:
file = "../DNS (Lee and Moser 2015)/Re - 550/LM_Channel_0550_RSTE_ww_prof.dat"    # file path is huge
ww_550_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
ww_550_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

ww_550_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,1.134836e-13,0.135829,-3.499842e-19,0.0,0.135829,-1.127727e-08
1,0.000005,0.002696,2.644295e-17,-4.336840e-12,0.135556,9.074049e-05,0.0,0.135647,-2.591633e-07
2,0.000025,0.013479,3.304088e-15,-5.360607e-10,0.134470,4.521470e-04,0.0,0.134923,-1.029280e-06
3,0.000069,0.037739,7.246826e-14,-1.189301e-08,0.132055,1.256292e-03,0.0,0.133313,-1.658833e-06
4,0.000149,0.080868,7.119109e-13,-1.190195e-07,0.127862,2.655542e-03,0.0,0.130518,-9.355451e-07
...,...,...,...,...,...,...,...,...,...
187,0.962515,523.123032,-2.850731e-08,6.615005e-04,0.000018,5.045685e-04,0.0,0.001184,2.873849e-07
188,0.970842,527.648902,-2.325801e-08,6.639025e-04,0.000019,4.966189e-04,0.0,0.001178,1.533003e-06
189,0.979172,532.175872,-1.730862e-08,6.665513e-04,0.000020,4.903998e-04,0.0,0.001174,3.270334e-06
190,0.987502,536.703628,-1.246403e-08,6.712070e-04,0.000021,4.861789e-04,0.0,0.001171,7.215811e-06


In [40]:
file = "../DNS (Lee and Moser 2015)/Re - 550/LM_Channel_0550_RSTE_uv_prof.dat"    # file path is huge
uv_550_df = pd.read_csv(
    file,
    sep='\\s+',
    comment='%',
    header=None
)
uv_550_df.columns = [
    'y/delta',
    'y^+',
    "Production",
    "Turbulent_Transport",
    "Viscous_Transport",
    "Pressure_Strain",
    "Pressure_Transport",
    "Viscous_Dissipation",
    "Balance"
]

uv_550_df

,y/delta,y^+,Production,Turbulent_Transport,Viscous_Transport,Pressure_Strain,Pressure_Transport,Viscous_Dissipation,Balance
0,0.000000,0.000000,0.000000e+00,4.854296e-15,-1.092520e-10,0.100540,-0.100540,4.519837e-20,-1.088980e-10
1,0.000005,0.002696,-7.394436e-15,-1.603541e-14,-1.577846e-05,0.100532,-0.100527,-1.051981e-05,-3.602515e-11
2,0.000025,0.013479,-4.606100e-12,-8.164532e-12,-7.904232e-05,0.100499,-0.100473,-5.271599e-05,-3.773617e-10
3,0.000069,0.037739,-2.810091e-10,-4.978017e-10,-2.222625e-04,0.100413,-0.100339,-1.483403e-04,-2.697790e-09
4,0.000149,0.080868,-5.847224e-09,-1.033796e-08,-4.798402e-04,0.100210,-0.100050,-3.206513e-04,-9.830339e-09
...,...,...,...,...,...,...,...,...,...
187,0.962515,523.123032,-4.624748e-04,-2.141744e-04,-5.235451e-07,0.000234,0.000397,-3.288609e-05,-1.265614e-05
188,0.970842,527.648902,-3.619815e-04,-1.695170e-04,-4.697694e-07,0.000180,0.000311,-2.580522e-05,-1.522167e-05
189,0.979172,532.175872,-2.600553e-04,-1.223657e-04,-2.588810e-07,0.000127,0.000222,-1.859104e-05,-1.495542e-05
190,0.987502,536.703628,-1.568390e-04,-7.341756e-05,-1.215650e-07,0.000076,0.000132,-1.123684e-05,-1.104338e-05


---

In [41]:
uu_550_df = uu_550_df.reset_index(drop=True)     # Ensure consistent ordering
vv_550_df = vv_550_df.reset_index(drop=True)
ww_550_df = ww_550_df.reset_index(drop=True)

epsilon_550 = (uu_550_df['Viscous_Dissipation'] + vv_550_df['Viscous_Dissipation'] + ww_550_df['Viscous_Dissipation'])  
epsilon_550_df = pd.DataFrame({        # Converting the series to DataFrame for merging it into the final Dataset
    'y^+': uu_550_df['y^+'] ,
    'epsilon': epsilon_550
})

In [42]:
vel_fluc_550_df = vel_fluc_550_df.sort_values('y^+').reset_index(drop=True)
b_11_550 = vel_fluc_550_df["u'u'"] / (2.0 * vel_fluc_550_df['k'] ) - 1.0 / 3.0  # All these four are Pandas Series
b_22_550 = vel_fluc_550_df["v'v'"] / (2.0 * vel_fluc_550_df['k']) - 1.0 / 3.0
b_33_550 = vel_fluc_550_df["w'w'"] / (2.0 * vel_fluc_550_df['k']) - 1.0 / 3.0
b_12_550 = vel_fluc_550_df["u'v'"] / (2.0 * vel_fluc_550_df['k'])

b_550_df = pd.DataFrame({                      # Converting the series to DataFrame for merging it into the final Dataset     
    'y^+': uu_550_df['y^+'],
    'b_11': b_11_550,
    'b_22': b_22_550,
    'b_33': b_33_550,
    'b_12': b_12_550
})

In [43]:
phi_550_df = pd.DataFrame({                      # Converting the series to DataFrame for merging it into the final Dataset     
    'y^+': uu_550_df['y^+'],
    'phi_11': uu_550_df['Pressure_Strain'],
    'phi_22': vv_550_df['Pressure_Strain'],
    'phi_33': ww_550_df['Pressure_Strain'],
    'phi_12': uv_550_df['Pressure_Strain']
})

---

In [44]:
df_550 = (
    mean_prof_550_df[['y/delta','y^+', 'U']]
    .merge(mean_prof_550_df[['y^+', 'dU/dy']], on='y^+')
    .merge(vel_fluc_550_df[['y^+', 'k']], on='y^+')
    .merge(vel_fluc_550_df[['y^+',"u'u'","v'v'","w'w'","u'v'"]], on='y^+')
    .merge(epsilon_550_df[['y^+', 'epsilon']], on='y^+')
    .merge(b_550_df[['y^+', 'b_11', 'b_22', 'b_33', 'b_12']], on='y^+')
    .merge(phi_550_df[['y^+','phi_11', 'phi_22', 'phi_33', 'phi_12']], on='y^+')
)

df_550

,y/delta,y^+,U,dU/dy,k,u'u',v'v',w'w',u'v',epsilon,b_11,b_22,b_33,b_12,phi_11,phi_22,phi_33,phi_12
0,0.000000,0.000000,0.000000,1.000000,2.203288e-34,1.458397e-34,0.000000e+00,2.948179e-34,0.000000e+00,0.463243,-0.002374,-0.333333,0.335707,0.000000,-1.208084e-19,4.707927e-19,-3.499842e-19,0.100540
1,0.000005,0.002696,0.002696,0.999995,8.414171e-07,1.189617e-06,7.394295e-15,4.932175e-07,-1.910621e-11,0.463023,0.373579,-0.333333,-0.040246,-0.000011,1.922750e-05,-1.099680e-04,9.074049e-05,0.100532
2,0.000025,0.013479,0.013478,0.999975,2.101485e-05,2.973260e-05,4.606104e-12,1.229710e-05,-2.390467e-09,0.462146,0.374085,-0.333333,-0.040752,-0.000057,9.539165e-05,-5.475386e-04,4.521470e-04,0.100499
3,0.000069,0.037739,0.037738,0.999931,1.643998e-04,2.329725e-04,2.810219e-10,9.582685e-05,-5.258512e-08,0.460206,0.375221,-0.333332,-0.041889,-0.000160,2.624249e-04,-1.518717e-03,1.256292e-03,0.100413
4,0.000149,0.080868,0.080862,0.999851,7.520443e-04,1.068751e-03,5.847957e-09,4.353315e-04,-5.193396e-07,0.456865,0.377231,-0.333329,-0.043901,-0.000345,5.447250e-04,-3.200267e-03,2.655542e-03,0.100210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.962515,523.123032,20.988490,0.001147,7.181116e-01,6.359682e-01,4.032170e-01,3.970381e-01,-3.616980e-02,0.003682,0.109473,-0.052585,-0.056887,-0.025184,-8.910478e-04,3.864793e-04,5.045685e-04,0.000234
188,0.970842,527.648902,20.993123,0.000900,7.148253e-01,6.319254e-01,4.023528e-01,3.953724e-01,-2.812336e-02,0.003666,0.108681,-0.051899,-0.056782,-0.019671,-8.737131e-04,3.770942e-04,4.966189e-04,0.000180
189,0.979172,532.175872,20.996627,0.000647,7.123300e-01,6.288644e-01,4.016998e-01,3.940958e-01,-2.008433e-02,0.003654,0.108080,-0.051371,-0.056709,-0.014098,-8.604931e-04,3.700933e-04,4.903998e-04,0.000127
190,0.987502,536.703628,20.998979,0.000391,7.106503e-01,6.268089e-01,4.012624e-01,3.932294e-01,-1.204935e-02,0.003647,0.107677,-0.051013,-0.056665,-0.008478,-8.516028e-04,3.654238e-04,4.861789e-04,0.000076


In [45]:
df_550.to_csv("../Datasets/Re-550.csv", index=False)    # Saving the DataFrame in the Datasets Folder (For model training)

# Finally, we have the four final datasets after doing all the required feature engineering and merging

---

# End of Notebook

## All the four .csv datasets are in the Datasets folder of this repository, look into it for the data insights

---